# Fracture deformation modeling

In this tutorial, we present the fracture deformation modeling capabilities in PorePy. In particular, we will visit:
- general modeling of fracture deformation and single model components
- non-smooth formulation of fracture deformation suitable for Newton's method
- tuning parameters for the nonlinear solution of fracture deformation

# General modeling 
Fracture deformation constraints the relative displacements of matrix blocks across a fracture. In the simplest case, it decomposes into normal and tangential directions.

The relevant variables are relative displacements $u$ (displacement jumps) and tractions $t$ across the fracture. The normal and tangential components are denoted by $n$ and $t$, respectively. The normal component is defined as the projection of the relative displacement onto the normal vector of the fracture. The tangential component is defined as the relative displacement minus the normal component.

The non-penetration condition encodes that the relative displacement in the normal direction is non-negative.
\begin{equation}
\tag{1}
-t_n \leq  0 \qquad \wedge \qquad [u]_n \geq g_n \qquad \wedge \qquad t_n ([u]_n - g_n) = 0
\end{equation}

The frictional sliding condition encodes that the tangential traction is bounded by a friction bound. If it is stricly less than the friction bound, the contact is in stick state (the tangential relative velocity is zero). If it is equal to the friction bound, the contact is in slip state (the tangential relative velocity is non-zero and aligns with the tangential traction).
\begin{equation}
\tag{2}
\|t_t\| \leq b_p \qquad \wedge \qquad \|t_t\| < b_p \Rightarrow [\dot{u}]_t = 0 \qquad \wedge \qquad \|t_t\| = b_p \Rightarrow \exists \xi>0: t_t = \xi [\dot{u}]_t
\end{equation}
While these two conditions serve as general framework for modeling contact, single components can be further detailed. In particular, displacement jumps, gaps and the friction bound allow for further modeling, added to your PorePy model by adding the right mixins. These are discussed separately below.

If no additional mixins are added, the above physical model is invoked. The associated material parameters of interest and to be used in the model setup are as follows:

## Friction bound
The default case, as part of `pp.MomentumBalance`, models Coulomb friction implemented as part of `pp.CoulombFrictionBound`. This defines $b_p$ as
\begin{equation}
b_p = F \cdot \mathrm{max}(-t_n, 0).
\end{equation}
where $F$ equals the `friction_coefficient` among `pp.SolidConstants`.

A respective model including Coulomb friction can be defined as follows:

In [ ]:
import porepy as pp

class MyMomentumBalance_1(pp.MomentumBalance):
    """Proxy class for the momentum balance."""

solid_constants = {"friction_coefficient": 0.5}
model_params = {
    "material_constants": {
        "solid": pp.SolidConstants(**solid_constants)
    }
}
model = MyMomentumBalance_1(model_params)

## Elastic-plastic split

One of the modeling capabilities in PorePy includes the split of the displacement jump into elastic and plastic components. In tangential direction this is modelled through the so-called Barton-Bandis model. 
\begin{equation}
...
\end{equation}


In [ ]:
class MyMomentumBalance_2(MyMomentumBalance_1):
    """MomentumBalance with elastic-plastic split."""

solid_constants.update({})
model_params = {
    "material_constants": {
        "solid": pp.SolidConstants(**solid_constants)
    }
}
model = MyMomentumBalance_2(model_params)

## Gap models
The gap $g_n$ between fractures provides means to incorporare effects as large-scale roughness leading to dilation or taking account for compressive strength of small-scale roughness leading to elastic displacement. A combined model for the gap $g_n$ is given by 
\begin{equation}
\tag{3}
g_n = g_\mathrm{ref} + g_\mathrm{shear} + g_\mathrm{elastic}.
\end{equation}
Only the first is active by default. The different inputs for the above model come as constants in `pp.SolidConstants`:
- $g_\mathrm{ref}$ is the constant reference fracture gap and equal to the solid constant `fracture_gap`.
- $g_\mathrm{shear}$ models dilation effects: $g_\mathrm{shear} = \mathrm{tan}(\theta_\mathrm{dil})\,\|u_{p,t}\|$ is controlled by the dilation angle $\theta_\mathrm{dil}$, equal to the solid constant `dilation_angle`.
- $g_\mathrm{elastic}$ models elastic deformation of fracture walls through the Barton-Bandis model: $g_\mathrm{elastic} = \Delta u_n^{max} + \frac{\Delta u_n^{max} t_n}{\Delta u_n^{max} K_n - t_n}$, where:
    - the maximum opening $\Delta u_n^{max}$ equals the solid constant `maximum_elastic_fracture_opening`
    - the fracture normal stiffness $K_n$ equals the solid constant `fracture_normal_stiffness`.

In [5]:
class MyMomentumBalance_2(MyMomentumBalance_1):
    ...
solid_constants.update({})
model_params = {
    "material_constants": {
        "solid": pp.SolidConstants(**solid_constants)
    }
}
model = MyMomentumBalance_2(model_params)

In [6]:
class MyMomentumBalance_2(MyMomentumBalance_1):
    ...
solid_constants.update({})
model_params = {
    "material_constants": {
        "solid": pp.SolidConstants(**solid_constants)
    }
}
model = MyMomentumBalance_2(model_params)

## Summary
Overall, we discussed how to control the general fracture deformation model implemented in PorePy. In summary, the essential inputs are controlled as components of `pp.SolidConstants`. In particular, the following code should be part of your model parameters to modify the above parameters.

In [7]:
import porepy as pp

model_parameters = {}
# Define your model parameters here.
...
# Define fracture deformation related parameters.
fracture_deformation_parameters = {
    "fracture_gap": 1e-3,
    "dilation_angle": 0.1,
    "maximum_elastic_fracture_opening": 3e-4,
    "fracture_normal_stiffness": 1e6,
    "friction_coefficient": 0.5,
}
model_parameters.update(
    {
        "solid": pp.SolidConstants(**fracture_deformation_parameters),
    }
)

# Let's have a look at the parameters.
print(model_parameters)


{'solid': SolidConstants(name='', units=<porepy.models.units.Units object at 0x000001C7D0F297C0>, constants_in_SI={'biot_coefficient': 1.0, 'density': 1.0, 'dilation_angle': 0.1, 'fracture_gap': 0.001, 'fracture_normal_stiffness': 1000000.0, 'fracture_tangential_stiffness': -1.0, 'friction_coefficient': 0.5, 'lame_lambda': 1.0, 'maximum_elastic_fracture_opening': 0.0003, 'normal_permeability': 1.0, 'permeability': 1.0, 'porosity': 0.1, 'residual_aperture': 0.1, 'shear_modulus': 1.0, 'skin_factor': 0.0, 'specific_heat_capacity': 1.0, 'specific_storage': 1.0, 'thermal_conductivity': 1.0, 'thermal_expansion': 0.0, 'well_radius': 0.1}, _initialized=True, biot_coefficient=1.0, density=1.0, dilation_angle=0.1, fracture_gap=0.001, fracture_normal_stiffness=1000000.0, fracture_tangential_stiffness=-1.0, friction_coefficient=0.5, lame_lambda=1.0, maximum_elastic_fracture_opening=0.0003, normal_permeability=1.0, permeability=1.0, porosity=0.1, residual_aperture=0.1, shear_modulus=1.0, skin_facto

# Nonlinear solution

In PorePy, the normal fracture deformation (1) and tangential fracture deformation (2) are encoded as semi-smooth equations to allow the use of a Newton strategy for the nonlinear solution. These are implented as `normal_fracture_deformation_equation` and `tangential_fracture_deformation_equation` as part of the class [pp.ContactMechanicsEquations](../src/porepy/models/contact_mechanics.py) and added to the set of equations, solved at each Newton iteration. 

# Tuning of nonlinear solution
There exist three important tuning parameters entering the fracture deformation equations:
- open state tolerance
- characteristic displacement
- characteristic contact traction
- units 

